<a href="https://colab.research.google.com/github/Jackbaude/slipv6/blob/main/slipv6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!apt install nmap
!sudo apt-get install build-essential cmake libgmp3-dev gengetopt libpcap-dev flex byacc libjson-c-dev pkg-config libunistring-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nmap is already the newest version (7.91+dfsg1+really7.80+dfsg1-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
libjson-c-dev is already the newest version (0.15-3~ubuntu1.22.04.2).
libjson-c-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libdbus-1-dev libfl-dev libfl2 libgmp-dev libgmpxx4ldbl libpcap0.8-dev
Suggested packages:
  bison flex-doc gmp-doc libgmp10-doc libmpfr-dev
The following packages will be REMOVED:
  pkgconf r-base-dev
The follow

In [ ]:
# prompt: !git clone https://github.com/idealeer/xmap.git
# then from the xmap repo
# cmake .
# make -j4

!git clone https://github.com/idealeer/xmap.git
%cd xmap
!cmake .
!make -j4


fatal: destination path 'xmap' already exists and is not an empty directory.
/content/xmap
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "0.29.2")
-- Checking for module 'json-c'
--   Found json-c, version 0.15
-- Default XMap configuration file location is /etc/xmap
-- Checking if blacklist4.conf exists there...
-- Checking if whitelist6.conf exists there...
-- Checking i

In [4]:
# prompt: open this in pandas https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/pops.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipaddress import IPv6Network, IPv4Network, IPv4Address

pops_url = "https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/pops.csv"
geoip_url = "https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/geoipstarlink.csv"

pops_df = pd.read_csv(pops_url)
geoip_df = pd.read_csv(geoip_url);

ipv4_df = geoip_df[geoip_df['ip_range'].str.contains(r'\.')]
ipv6_df = geoip_df[geoip_df['ip_range'].str.contains(r'\:')]

ipv4_df
ipv6_df

,ip_range,country code,region_code,city,Unnamed: 4
1731,2a0d:3340:f400::/38,DE,DE-BE,Berlin,NaN
1732,2406:2d40:2000::/40,AU,AU-WA,Perth,NaN
1733,2406:2d40:3000::/40,JP,JP-13,Tokyo,NaN
1734,2406:2d40:3300::/40,MN,MN-1,Ulaanbaatar,NaN
1735,2406:2d40:3400::/40,PH,PH-00,Manila,NaN
...,...,...,...,...,...
2213,2605:59c7:f009::/48,US,US-TX,Starbase,NaN
2214,2605:59c7:f080::/48,US,US-FL,Miami,NaN
2215,2605:59c7:f081::/48,US,US-FL,Miami,NaN
2216,2605:59c7:f0c0::/48,US,US-FL,Miami,NaN


In [5]:
def generate_starlink_addresses(prefixes, subprefix_length=56):
    """
    Generate IPv6 addresses for the given prefixes and return them as a list.
    :param prefixes: List of base prefixes (e.g., ['2605:59ca:1300::/40'])
    :param subprefix_length: Subnet size to enumerate (default is /56)
    :return: List of generated IPv6 addresses
    """
    addresses = []
    for base_prefix in prefixes:
        base_network = IPv6Network(base_prefix)
        print(f"Generating addresses for {base_prefix}")
        # Generate all /56 sub-prefixes
        for subprefix in base_network.subnets(new_prefix=subprefix_length):
            # Fix the 57th to 127th bits to 0, 128th bit to 1
            addr = f"{subprefix.network_address}1"
            addresses.append(addr)
    return addresses



In [9]:
# Extract IPv6 prefixes from the DataFrame
ipv6_prefixes = ipv6_df['ip_range'].tolist()[:5]

prefixes = ['2605:59ca:1300::/40', '2605:59ca:1600::/40']

# Generate all possible IPv6 addresses
all_ipv6_addresses = generate_starlink_addresses(prefixes)

# write these ip addrs to a file
with open('addresses.txt', 'w') as f:
    for ip in all_ipv6_addresses:
        f.write(ip + '\n')

Generating addresses for 2605:59ca:1300::/40
Generating addresses for 2605:59ca:1600::/40


In [10]:
# Step 2: Use nmap to scan the addresses
!nmap -v -6 -iL addresses.txt -oN starlink_scan_results.txt

Streaming output truncated to the last 5000 lines.
setup_target: failed to determine route to 2605:59ca:16ec:7c00::1
setup_target: failed to determine route to 2605:59ca:16ec:7d00::1
setup_target: failed to determine route to 2605:59ca:16ec:7e00::1
setup_target: failed to determine route to 2605:59ca:16ec:7f00::1
setup_target: failed to determine route to 2605:59ca:16ec:8000::1
setup_target: failed to determine route to 2605:59ca:16ec:8100::1
setup_target: failed to determine route to 2605:59ca:16ec:8200::1
setup_target: failed to determine route to 2605:59ca:16ec:8300::1
setup_target: failed to determine route to 2605:59ca:16ec:8400::1
setup_target: failed to determine route to 2605:59ca:16ec:8500::1
setup_target: failed to determine route to 2605:59ca:16ec:8600::1
setup_target: failed to determine route to 2605:59ca:16ec:8700::1
setup_target: failed to determine route to 2605:59ca:16ec:8800::1
setup_target: failed to determine route to 2605:59ca:16ec:8900::1
setup_target: failed to d